# Импорт библиотек.

In [202]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from pymorphy2 import MorphAnalyzer
from sklearn import metrics #метрики
from sklearn.metrics import accuracy_score

from gensim.models import Word2Vec

import annoy


# Просмотр краткого содержания по данным.

In [160]:
df1 = pd.read_csv('data/ProductsDataset.csv')
df1=df1.loc[:, ['title', 'product_id']]
df1["label"] = np.ones(len(df1), dtype=int)
df1.head()

,title,product_id,label
0,Юбка детская ORBY,58e3cfe6132ca50e053f5f82,1
1,Ботильоны,5667531b2b7f8d127d838c34,1
2,Брюки,59534826aaab284cba337e06,1
3,Продам детские шапки,57de544096ad842e26de8027,1
4,Блузка,5ad4d2626c86cb168d212022,1


Проверим наличие дубликатов

In [167]:
df1_dupl=df1[df1.duplicated(['product_id'])]

In [168]:
df1_dupl

,title,product_id,label
4650,размер 52-54,NaN,1
4651,взади сетка,NaN,1
4652,----------,NaN,1
4653,Могу выслать в любой город Р.Ф.,NaN,1
26827,"Лампа галогенная GU5.3,GU10 MR16 20,35,50,75Вт",NaN,1
26828,117шт*15р,NaN,1
27016,Кепка Minnie Mouse C прямым козырьком размер 5...,NaN,1
27017,C прямым козырьком,NaN,1
27018,Размер 50-54,NaN,1
27019,----------,NaN,1


In [171]:
df1.shape

(35536, 3)

Возьму ровно столько же строк )

In [175]:
df2 = pd.read_csv('data/prepared_answers.txt', usecols=[0],sep="\t",nrows=35536).rename(columns={"Unnamed: 0":"title"})
df2["label"] = np.zeros(len(df2), dtype=int)
df2.sample(4)

,title,label
19929,Ты хитренький?)) .,0
11271,фильмы типа братз .,0
12798,"посоветуйте клевый триллер:))) мне кажется, чт...",0
24243,Когда короли завидуют своим шутам(ну кроме жен...,0


In [176]:
data = pd.concat([df1, df2])

In [177]:
data.sample(5)

,title,product_id,label
700,А вот У МЕНЯ ТАКОЙ ВОТ ВОПРОС ЕСТЬ!! ! АВТОМОБ...,NaN,0
34425,Норковая шуба,5943bb679380007d657c43e2,1
17979,Туфли,599e779bf20263cb594cce5c,1
469,Написать маленькое сочинение на тему отдых дет...,NaN,0
25388,Спецодежда,5aeac56d22a44938b31d9c72,1


## Осуществлён препроцессинг текста (как минимум удаление знаков препинания, приведение к нижнему регистру, стемминг/лемматизация).

In [179]:
import string
def clean_text(text):
    # пунктуацию будем удалять в цикле
    for p in string.punctuation:
        if p in text:
            text = text.replace(p, ' ')
    text = text.lower()  # приведение к нижнему регистру
    text = text.strip()  # удаление лишних пробелов с обеих сторон
    words = text.split()  # разбиение текста на слова
    return words

In [180]:
data['title']

0                                        Юбка детская ORBY
1                                                Ботильоны
2                                                    Брюки
3                                     Продам детские шапки
4                                                   Блузка
                               ...                        
35531    как мужик двух генералов накормил Как описывае...
35532    Как можно заработать БОЛЬШЕ денег - за МЕНЬШЕЕ...
35533    выключается видеокарта во время игр. выключает...
35534    Ребят!! Нашёл удочку дома))) подскажите сайт, ...
35535       зачем нужны виртуальные части тела?))))))))) .
Name: title, Length: 71072, dtype: object

In [181]:
data['title'] = data['title'].astype(str)
data['title'] = data['title'].apply(clean_text)

In [182]:
data['title']

0                                    [юбка, детская, orby]
1                                              [ботильоны]
2                                                  [брюки]
3                                 [продам, детские, шапки]
4                                                 [блузка]
                               ...                        
35531    [как, мужик, двух, генералов, накормил, как, о...
35532    [как, можно, заработать, больше, денег, за, ме...
35533    [выключается, видеокарта, во, время, игр, выкл...
35534    [ребят, нашёл, удочку, дома, подскажите, сайт,...
35535             [зачем, нужны, виртуальные, части, тела]
Name: title, Length: 71072, dtype: object

In [183]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/SEV/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [184]:
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('russian')


In [185]:
def remowe_stopwords(tokenised_list):
    text = [word for word in tokenised_list if word not in stopwords]
    return text

data['title'] = data['title'].apply(lambda x: remowe_stopwords(x))

In [186]:
data

,title,product_id,label
0,"[юбка, детская, orby]",58e3cfe6132ca50e053f5f82,1
1,[ботильоны],5667531b2b7f8d127d838c34,1
2,[брюки],59534826aaab284cba337e06,1
3,"[продам, детские, шапки]",57de544096ad842e26de8027,1
4,[блузка],5ad4d2626c86cb168d212022,1
...,...,...,...
35531,"[мужик, двух, генералов, накормил, описывается...",NaN,0
35532,"[заработать, денег, меньшее, время]",NaN,0
35533,"[выключается, видеокарта, время, игр, выключае...",NaN,0
35534,"[ребят, нашёл, удочку, дома, подскажите, сайт,...",NaN,0


In [187]:
#лемматизация
morpher = MorphAnalyzer()
def lem_words(text):
    text_morph = [morpher.parse(i)[0].normal_form for i in text]
    return text_morph

In [188]:
data['title']= data['title'].apply(lambda x: lem_words(x))


In [189]:
data.sample(10)

,title,product_id,label
9882,"[ползунок, детский, пакет, 0]",5b1127b1cf689a356b549d42,1
17005,"[отличие, dvd, r, dvd, r, dvd, rw, dvd, rw]",NaN,0
9101,"[помочь, пожалуйста, котёнок, трое, месячный, ...",NaN,0
1972,"[какой, учереждение, лпу, государственный, ком...",NaN,0
13272,"[джинсы, турция]",5ae048f7bd36c0125a619ab5,1
15256,[кофта],5aeac8b82138bbbc23492ea2,1
2577,"[фирменный, топ, fendi]",588773069a64a299b242016c,1
22233,"[отличаться, диск, dvd, rw, dvd, rw]",NaN,0
480,"[четыре, сословие]",NaN,0
7124,"[штаны, утеплить, 110р, цена, 2, пара]",591ae42b2756ba58431fe233,1


In [190]:
print('{} - количество пропусков в text'.format(data['label'].isnull().sum()))
print('{} - количество пропусков в label'.format(data['label'].isnull().sum()))


0 - количество пропусков в text
0 - количество пропусков в label


## Текст векторизирован любым из изученных способов (CountVectorizer, TfidfVectorizer, HashingVectorizer, Word2Vec).

In [191]:
X = data['title']
Y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X,Y, train_size=0.7,random_state=42, shuffle=True)

In [192]:
print(X_train.shape)
print(X_test.shape)

(49750,)
(21322,)


In [193]:
# Join the words in each list into a single string
X_train_processed = [' '.join(doc) for doc in X_train]
X_test_processed = [' '.join(doc) for doc in X_test]

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer(max_features= 30000)



In [255]:
X_train_v = vectorizer.fit_transform(X_train_processed)

X_test_v = vectorizer.transform(X_test_processed)

In [256]:
print(X_train_v.shape)
print(X_test_v.shape)

(49750, 30000)
(21322, 30000)


In [257]:
model = LogisticRegression()
model.fit(X_train_v, y_train)

LogisticRegression()

In [258]:
predictions = model.predict(X_test_v)
accuracy = (predictions == y_test).mean()
accuracy

0.9748147453334584

In [259]:
accuracy_score(y_test, predictions)

0.9748147453334584

In [222]:
# Обучим модель word2vec на продуктах
mask = (data['label'] == 1)
sentences = data[mask]['title']
model_w2v_prod = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=10)
model_w2v_prod.save("w2v_model")

In [226]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}

for row in data.itertuples():
    n_w2v = 0
    index_map[row[0]] = row[0]
    question = row[1]
    vector = np.zeros(100)
    for word in question:
        if word in model_w2v_prod.wv:
            vector += model_w2v_prod.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index.add_item(row[0], vector)
        

index.build(10)
index.save('product_ann.ann')

True

In [228]:


# Assuming model_w2v_prod is your Word2Vec model
#model_w2v_prod = Word2Vec.load("w2v_model")

# Specify the dimension of the vectors in the Word2Vec model
vector_dim = model_w2v_prod.vector_size

# Build the Annoy index
indexer = annoy.AnnoyIndex(vector_dim, 'angular')
for i, word in enumerate(model_w2v_prod.wv.index_to_key):
    indexer.add_item(i, model_w2v_prod.wv[word])
indexer.build(50)  # Build the index with 50 trees for better accuracy

def find_similar_products(query, index, model_w2v_prod):
    query_vector = np.zeros(100)
    n_w2v=0
    for word in query:
        if word in model_w2v_prod.wv:
            query_vector += model_w2v_prod.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        query_vector = query_vector / n_w2v
        
    similar_docs = indexer.get_nns_by_vector(query_vector, n=5)  # Find 5 most similar products
    return similar_docs

# Example usage
query = ['юбка','красная','короткая']
similar_products = find_similar_products(query, indexer, model_w2v_prod)
print(similar_products)


[513, 300, 991, 778, 505]


Загрузим индекс,словарь и модель  из прошлого юниту ChatBotPractic.

In [89]:
# Создание нового объекта AnnoyIndex
index = annoy.AnnoyIndex(100, 'angular')

# Загрузка сохраненного индекса из файла 'speaker.ann'
index.load('speaker.ann')

# Теперь индекс готов к использованию в  программе


True

In [90]:
model_w2v_sp= Word2Vec.load("w2v_model_sp")

In [91]:
import pickle
 
with open('my_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [ ]:
loaded_dict

{0: 'вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . \n',
 1: 'меня вобще прикалывает эта тема :). \n',
 2: 'Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. Выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. \n',
 3: 'ЭБУ — электронный блок управления двигателем автомобиля, его другое название — контроллер. Он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы.. \n'

In [289]:
def preprocess_txt(line):
    line = " ".join(str(element) for element in line)
    line = clean_text(line)
    remowe_stopwords(line)
    lem_words(line)
    return line

In [93]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model_w2v_sp.wv:
            vector += model_w2v_sp.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return loaded_dict[answer_index[0]]

In [305]:
def get_answer(text_input):
    #
    #print('test_prod',test_prod)
    predict = model.predict(vectorizer.transform(text_input))
    print('predict',predict)
    if predict == 0:

        print('Ответ на ваш вопрос:',find_answer(text_input))
    else:
        test_prod = preprocess_txt(text_input)
        find_similar_products(text_input, indexer, model_w2v_prod)
        print('Обратите внимание на след. продукты:')
        for el in similar_products:
            print('Название -', df1['title'].iloc[el],', id-',df1['product_id'].iloc[el])

In [303]:
get_answer(['юбка красная короткая'])

predict [1]
Обратите внимание на след. продукты:
Название - Новый кошелек , id- 584d1969479e757c213e9366
Название - Кофта на мальчика , id- 58fbf86c9e94ba78b77dfec2
Название - Рубашка , id- 57ce5eecd53f3d0e6f856060
Название - Очки Cartier Edition Santos Dumont , id- 59dbbd2922a449a8e6351292
Название - Парка для девочки John Lewis , id- 5acdc463f094f35087394661


In [311]:
get_answer(['сколько солдат в роте'])

predict [0]
Ответ на ваш вопрос: а я почем знаю. 

